<a href="https://colab.research.google.com/github/CRAFTYPROGRAMMER826/IITGN-SRIP2025-IP0NB0000020/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O "dataset.zip" "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"

--2025-03-02 08:15:36--  https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataset.zip’

dataset.zip             [      <=>           ]  58.18M  11.9MB/s    in 12s     

2025-03-02 08:15:50 (4.74 MB/s) - ‘dataset.zip’ saved [61005872]



In [2]:
!unzip "dataset.zip" -d "/content/"

Archive:  dataset.zip
 extracting: /content/UCI HAR Dataset.names  
 extracting: /content/UCI HAR Dataset.zip  


In [3]:
!unzip "UCI HAR Dataset.zip" -d "/content/UCI HAR/"

Archive:  UCI HAR Dataset.zip
   creating: /content/UCI HAR/UCI HAR Dataset/
  inflating: /content/UCI HAR/UCI HAR Dataset/.DS_Store  
   creating: /content/UCI HAR/__MACOSX/
   creating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: /content/UCI HAR/UCI HAR Dataset/activity_labels.txt  
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._activity_labels.txt  
  inflating: /content/UCI HAR/UCI HAR Dataset/features.txt  
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._features.txt  
  inflating: /content/UCI HAR/UCI HAR Dataset/features_info.txt  
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._features_info.txt  
  inflating: /content/UCI HAR/UCI HAR Dataset/README.txt  
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._README.txt  
   creating: /content/UCI HAR/UCI HAR Dataset/test/
   creating: /content/UCI HAR/UCI HAR Dataset/test/Inertial Signals/
  inflating: /content/UCI H

In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
# importing the necessary libraries

In [46]:
X_train_path = "/content/UCI HAR/Dataset/train/X_train.txt"
y_train_path = "/content/UCI HAR/Dataset/train/y_train.txt"
X_test_path = "/content/UCI HAR/Dataset/test/X_test.txt"
y_test_path = "/content/UCI HAR/Dataset/test/y_test.txt"

#defining the paths of the trainning and the test data files along with the lables
print(y_train.shape)

X_train = np.loadtxt(X_train_path)
y_train = np.loadtxt(y_train_path).astype(int).ravel()
X_test = np.loadtxt(X_test_path)
y_test = np.loadtxt(y_test_path).astype(int).ravel()
print(y_train.shape)

# converting the x train and x test files to a numpy array as they only contain the numerical values
# and the processing of a numpy array is faster than a pandas dataframe

# we also convert the y train and y test files to type integers just in case there ois andy decimal value in the lables
#  along with that we use the ravel function to convert the lables to only 1 d format which is accepted by the lstm model

(7352,)
(7352,)


In [47]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# applying the sandard scaler method from the scikit library
# using Z score normalization scaling method for making the data centered around
# zero and reducing the diverse range of values (X scaled = ([X- meaan]/ Standard Deviation)

In [48]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
#originally the x test and train files were in a two  dimensional format.
# but we convert them to a three dimensional format because thew lstm model is generally
# used to predict real time data so it expects three dimensional data
# each dimension corresponds to format(lables, timesteps , features)
# lables denote the type of outcome to be predicted here it is the type of activity
# timesteps denote the  number of samples for which the features would be evaluated and the lables would be predicted
# here we don't have any real time data so we use timesteps =1 which suggests that features of all the samples would be
# evaluated in a single iteration and the p[redictions would be generated in a single teration too
#

In [49]:
y_train = to_categorical(y_train - 1)  # Adjust labels to start from 0 because the traditional deep learning models
# expect the lables to start from zero and the y train and test files originally had lables from 1 to 6
y_test = to_categorical(y_test - 1)

In [50]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')
])
# using the sequential function from keras package of tensorflow library
# creating layer of lstm neural network with 64 neurons with shape (timesteps,features) by excluding the lables
# return_sequences=True  Ensures the output is a sequence and not the pattern detected by final state
#  (needed for stacking multiple LSTMs)
# but it does not make any difference her because we only have 1 timestep so true and false will behave the same
# droupout=0.2 suggests 20% neurons would be randomly switched off to prevent overfitting of data
# another layer of 32 neurons to refine and process the pattern analysed by first layer it also follows dropout= 0.2
# the dense layer has  6 neurons each correspondingnto one of the lables of the dataset
# enabling softmax causes each neuron to generate probabilities for the occurance of their respective lables based
# test data features
print(y_train.shape)


(7352, 6)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Adam (Adaptive Moment Estimation) Accuracy= Total Samples / Correct Predictions

In [52]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)
# the model checks the entire dataset 20 times  checking 32 samples at once and checks from seperate test data
# and the weights are reset once again by backpropagation and the process goes on

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6928 - loss: 1.0464 - val_accuracy: 0.9379 - val_loss: 0.1842
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9514 - loss: 0.1476 - val_accuracy: 0.9393 - val_loss: 0.1716
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9745 - loss: 0.0850 - val_accuracy: 0.9342 - val_loss: 0.1955
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9733 - loss: 0.0777 - val_accuracy: 0.9372 - val_loss: 0.1836
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9806 - loss: 0.0609 - val_accuracy: 0.9423 - val_loss: 0.1779
Epoch 6/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9760 - loss: 0.0702 - val_accuracy: 0.9382 - val_loss: 0.2063
Epoch 7/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9830 - loss: 0.0483 - val_accuracy: 0.9369 - val_loss: 0.2096
Epoch 8/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9832 - loss: 0.0477 - val_accuracy: 0.

In [58]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {accuracy:.4f}")
print(f"losses:{loss:.4f}")

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9392 - loss: 0.1987
LSTM Model Accuracy: 0.9410
losses:0.2273
